In [ ]:
import { useEffect, useState } from "react";
import { useParams } from "react-router-dom";
import "./Components/styles/searchPageStyle.css";
import MovieCard from "./Components/movieCard";
import NavBar from "./Components/NavBar";
import ReactPlayer from "react-player";

const SearchPage = () => {
    const params = useParams();
    const apiKey = "api_key=7212cc714eacf62263334b404a1fc587";
    const inputValue = params.id; // retrieving the searched movie name
    const [searchedMovie, setSearchedMovie] = useState({});
    const [recommendedMovies, setRecommendedMovies] = useState([{}]);
    const [director, setDirector] = useState(""); 
    const [castMembers, setCastMembers] = useState([{}]);
    const [genreList, setGenreList] = useState([{}]);
    const [currGenre, setCurrGenre] = useState([{}]);
    const [videoData, setVideoData] = useState([]);
    const [playTrailer, setPlayTrailer] = useState(0);
    const gotCast = (castData) => {
        setCastMembers([]);

        let counter = 3;
        for (let cast of castData) {
            setCastMembers((castMembers) => [...castMembers, cast]);
            counter--;
            if (counter === 0) break;
        }
    };
    const gotVideo = (data) => {
        if (data.videos && data.videos.results) {
            // Cherche une vidéo avec la langue française
            const frenchTrailer = data.videos.results.find(
                (vid) => vid.language === "fr" && vid.name === "Official Trailer"
            );
    
            // Si une bande-annonce en français existe, utilise-la
            setVideoData(frenchTrailer ? frenchTrailer : data.videos.results[0]);
        }
    };
    
    const gotRecommendedData = (apiData) => {
        setRecommendedMovies([]);
        let counter = 8;
        // getting data for each of the recommened movies
        for (let movie of apiData.movies) {
            fetch(
                `https://api.themoviedb.org/3/search/movie?${apiKey}&query=${movie}&language=fr`
            ).then((Response) =>
                Response.json().then((data) =>
                    setRecommendedMovies((recommendedMovies) => [
                        ...recommendedMovies,
                        data.results[0],
                    ])
                )
            );
            counter--;
            if (counter === 0) break;
        }
    };
    useEffect(
        () => {
            const gotTMDBData = (apiData) => {
                const realMovieData = apiData.results[0];
                setCurrGenre([]); 
                setCurrGenre(realMovieData.genre_ids);
    
                setSearchedMovie(realMovieData);
    
                // Fetch credits to get the cast and director
                fetch(
                    `https://api.themoviedb.org/3/movie/${realMovieData.id}/credits?${apiKey}`
                ).then((Response) =>
                    Response.json().then((data) => {
                        gotCast(data.cast);
    
                        // Find the director in the crew
                        const director = data.crew.find(member => member.job === "Director");
                        if (director) {
                            setDirector(director.name); // Affiche le nom du réalisateur
                        }
                    })
                );
    
                // Fetch movie details (with videos)
                fetch(
                    `https://api.themoviedb.org/3/movie/${realMovieData.id}?${apiKey}&append_to_response=videos&language=fr`
                ).then((Response) =>
                    Response.json().then((data) => gotVideo(data))
                );
            };
    
            // Getting data for the searched movie from TMDb
            fetch(
                `https://api.themoviedb.org/3/search/movie?${apiKey}&query=${inputValue}&language=fr`
            ).then((Response) =>
                Response.json().then((data) => gotTMDBData(data))
            );
    
            // Fetching recommended movies
            fetch(`/api/similarity/${inputValue}`).then((Response) =>
                Response.json().then((data) => gotRecommendedData(data))
            );
    
            // Fetching the list of genres
            fetch(
                `https://api.themoviedb.org/3/genre/movie/list?${apiKey}&language=fr`
            ).then((Response) =>
                Response.json().then((data) => setGenreList(data.genres))
            );
        },
        [inputValue] /*Making api call whenever the searched movie changes */
    );
    
    const RenderMovies = () =>
        recommendedMovies.map((movie) => {
            if (movie) {
                return (
                    <MovieCard
                        key={movie.id + movie.original_title}
                        movie={movie}
                    />
                );
            } else {
                return null;
            }
        });
        const RenderTrailer = () => {
            return (
                <div>
                    <ReactPlayer
                        url={`https://www.youtube.com/watch?v=${videoData.key}-U`} // Lien vers la bande-annonce
                        playing={true}
                        width="100%"
                        height="100%"
                        controls={true}
                        className="youtube-container"
                    />
                </div>
            );
        };
        const displayGenre = () =>
            currGenre.map((movieId, ind) => {
                if (ind >= 3) return null;
                if (movieId) {
                    for (let obj of genreList) {
                        if (obj.id === movieId) {
                            if (ind === 2) {
                                return <span>{obj.name}</span>;
                            } else {
                                return (
                                    <span>
                                        {obj.name}
                                        {","}{" "}
                                    </span>
                                );
                            }
                        }
                    }
                } else {
                    return null;
                }
                return null;
            });
    
        const imgLink = "https://image.tmdb.org/t/p/original";
        const backdropPath = "https://image.tmdb.org/t/p/w1280";
        return (
            <div
                style={{
                    backgroundImage: `linear-gradient(to bottom, rgba(0, 0, 0, 0), rgba(0, 0, 0, 1)), url(${backdropPath}${searchedMovie.backdrop_path})`,
                }}
                className="MainBackGround"
            >
                <NavBar isHome={true} />
    
                <div className="container trailerContainer">
                    {
                        videoData && playTrailer
                            ? RenderTrailer()
                            : null /*Rendering the trailer*/
                    }
                    <div className="container .movie-details">
                        <div className="row ">
                            <div className="col-md-6 left-box col-md-push-6">
                                <h1 className="topTitle-Movie">
                                    {searchedMovie.title}{" "}
                                </h1>
                                <p className="overviewContent">
                                {searchedMovie.overview}
                                </p>
                                <p>Distribution: </p>
                                <div className="casting">
                                {/* Affiche le nom du réalisateur en premier */}
                                    {director && (
                                    <p><b>Réalisateur:</b> {director}</p>
                                    )}

                                {/* Affiche les noms des acteurs */}
                                    {castMembers.map((member) => {
                                        if (member) {
                                            return (
                                            <p key={member.cast_id + member.id}>
                                                <a
                                                    href={`https://en.wikipedia.org/wiki/${member.name}`}
                                                    target="_blank"
                                                    rel="noreferrer"
                                                >
                                                {member.name}
                                                </a>
                                            </p>
                                            );
                                        }
                                        return null;
                                    })}
                                </div>

                                <div>
                                <b>Score d'évaluation{" : "}</b>
                                {searchedMovie.vote_average}
                                {"/10 "}

                                <i className="fa-solid fa-star"></i>
                            </div>
                            <div>
                                <b> Date de Sortie</b>
                                {" : "} {searchedMovie.release_date}
                            </div>
                            <div>
                                <b>Genres</b>
                                {" : "}
                                {currGenre ? displayGenre() : null}
                            </div>
                            <div>
                                <button
                                    className="trailer-bttn "
                                    onClick={() => setPlayTrailer(true)}
                                >
                                    <i className="fa-solid fa-play"></i>
                                    {"Regarder la bande d'annonce"}
                                </button>
                            </div>
                        </div>
                        <div className="col-md-6 col-md-pull-6 text-center">
                            <img
                                className="main-img"
                                src={`https://image.tmdb.org/t/p/w500${searchedMovie.poster_path}`}
                                alt="Movie"
                            />
                        </div>
                    </div>
                </div>
            </div>
            {/*Trailer Close Button */}
            <div className={playTrailer ? "DisplayOn" : "DisplayOFF"}>
                <button
                    className="close-bttn"
                    onClick={() => setPlayTrailer(false)}
                >
                    Close Trailer
                </button>
            </div>

            <div className="container-fluid recommendedMovies">
                <h2 className=" container RecommendHeading">
                    Recommended Movies
                </h2>
                {/*Rendering the recommended movie cards */}
                <div className="container recommendedGrid">
                    {RenderMovies()}
                </div>
            </div>
        </div>
    );
};

export default SearchPage;

                                
        

In [ ]:
import SearchBar from "./Components/SearchBar";
import NavBar from "./Components/NavBar";
import "./Components/styles/mainHomepageStyle.css";
import { useEffect, useState } from "react";

const MainHomepage = () => {
    const [list, setList] = useState([]);
    const [currMovies, setCurrMovies] = useState([{}]);
    useEffect(() => {
        setCurrMovies([]);
        setList([]);
        //getting the list of all movies from our flask server for our searchbar
        fetch("/api/movies").then((Response) =>
            Response.json().then((data) => setList(data.arr))
        );
    }, []);

    return (
        <div className="container-fluid">
            <div className="HomePage">
                <NavBar isHome={false} />
                <div className="HomeSearch">
                    {/*Rendering the searchbar */}
                    <SearchBar movies={list} placeholder="Cherchez un film" />
                </div>
            </div>
        </div>
    );
};

export default MainHomepage;

.HomeSearch {
    margin-top: 20vh;
}
.genreHeader {
    color: #fff !important;
    text-align: center;
    font-family: "Merriweather";
    margin-top: 100px;
}
.buttonGrid {
    display: flex;
    justify-content: center;
    flex-wrap: wrap;
    align-items: center;
    margin: 0 auto;
    margin-top: 2px;
    margin-bottom: 0px !important;
    padding-bottom: 20px;
    width: 60%;
    margin-bottom: 6.5vh;
    transform: 0.2s all;
}

.HomeFooter {
    position: "fixed";
    left: 0;
    bottom: 0;
    right: 0;
}

.genreTagON {
    padding: 10px 20px;
    color: black !important;
    background-color: #edf2f4;
    border-radius: 20px;
    margin-left: 3px;
    margin-right: 3px;
    cursor: pointer;
    margin: 4px 5px;
}
.genreTagON:hover {
    background-color: #adb5bd !important;
}
.genreTagOFF {
    color: #fff !important;
    padding: 10px 20px;
    background-color: #ef233c;
    border-radius: 20px;
    margin-left: 3px;
    margin-right: 3px;
    cursor: pointer;
    margin: 4px 5px;
}
.genreTagOFF:hover {
    background-color: #a4161a !important;
}
.container-fluid {
    padding: 0;
}
i {
    padding-left: 4px;
}
.HomeMovies {
    width: 100%;
    background-color: #0f1014;
    margin-top: 0;
    padding-top: 30px;
}
.HomeMovieGrid {
    display: grid;
    gap: 15px;
    grid-template-columns: 1fr 1fr 1fr 1fr;
}

@media screen and (max-width: 560px) {
    .HomePage {
        height: 100%;
        background-size: cover;
        background-color: #0f1014;
    }
}
@media screen and (max-width: 1080px) {
    .HomeMovieGrid {
        gap: 20px;
        grid-template-columns: 1fr 1fr 1fr;
    }
}
@media screen and (max-width: 900px) {
    .HomeMovieGrid {
        gap: 12px;
        grid-template-columns: 1fr 1fr 1fr;
    }
}

@media screen and (max-width: 770px) {
    .HomeMovieGrid {
        gap: 20px;
        grid-template-columns: 1fr 1fr;
    }
    .genreHeader {
        font-size: 20px;
    }
    .genreTagOFF {
        font-size: 14px;
        padding: 5px 10px;
        margin-left: 2px;
        margin-right: 2px;
    }
    .genreTagON {
        font-size: 14px;
        padding: 5px 10px;
        margin-left: 2px;
        margin-right: 2px;
        margin: 3px 5px;
    }
}

@media screen and (max-width: 560px) {
    .HomeMovieGrid {
        gap: 9px;
        grid-template-columns: 1fr 1fr;
    }
}